# **Capstone Project - The Battle of the Neighborhoods**

### Applied Data Science Capstone by IBM/Coursera

## **Table of contents** 


* [Introduction: Business Problem]()
* [Data]()
* [Methodology]()
* [Analysis]()
* [Results and Discussion]()
* [Conclusion]()




##  **Introduction: Business Problem**

This project aims to select the **Unsafe District** in San Francisco based on the **total crimes**, explore the **District´s** and finally cluster them using **k-mean clustering**.

This report will be targeted to people who are looking to **relocate to San Francisco**. Inorder to finalise a Disctrict, safety is considered as a top concern when moving to a new place. If you don’t feel safe in your own home, you’re not going to be able to enjoy living there. The crime statistics will provide an insight into this issue.

We will focus on the **unsafe districts** and explore **the total number of crimes** as well **the 5 most unsafe districts** and **the total crimes per category**, in that way the people can visualize the kind of insecurity per district.

## **Data** 

Based on definition of our problem, factors that will influence our decision are:

* The total number of crimes commited in each of the District.
* The 5 most unsafe districts.
* The total crimes per category.

Following data sources will be needed to extract/generate the required information:

* [Part 1: Preprocessing a real world data set from Kaggle showing the San Francisco Crimes:]() A dataset consisting of the crime statistics of each District in San Francisco obtained from Kaggle



* [Part 2: Creating a new dataset of the Districts of the unsafe Districts in San Francisco and generating their co-ordinates.:]() Co-ordinate of District will be obtained using Google Maps **API geocoding**

In [41]:
!pip install beautifulsoup4
#-c conda-forge
#!conda install -c anaconda beautifulsoup4


print("[yes]")


     |████████████████████████████████| 122kB 5.2MB/s eta 0:00:01
[yes]


In [42]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from bs4 import BeautifulSoup # library for web scrapping  

!conda install -c conda-forge geocoder --yes
import geocoder

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotlipy-0.7.0             |py36h8c4c3a4_1000         346 KB  conda-forge
    chardet-3.0.4              |py36h9f0ad1d_1006         188 KB  conda-forge
    click-7.1.2                |     pyh9f0ad1d_0          64 KB  conda-forge
    cryptography-2.9.2         |   py36h45558ae_0         613 KB  conda-forge
    future-0.18.2              |   py36h9f0ad1d_1         714 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    pysocks-1.7.1              |   py36h9f0ad1d_1          27 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    requests-2.24.0            |     pyh9f0ad1d_0          47 KB  conda-forge
    -

In [44]:
CLIENT_ID = 'KEM2IH2NSIW0RGMJT4N32DUHCUM5QCZGYYYIJOSVGF1INRWX' # your Foursquare ID
CLIENT_SECRET = 'AYJMMOUCT4NCRONQF13ZB15SVLX4ZRUUNY0NTGZCF04SKYVS' # your Foursquare Secret

VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentails:
CLIENT_ID: KEM2IH2NSIW0RGMJT4N32DUHCUM5QCZGYYYIJOSVGF1INRWX
CLIENT_SECRET:AYJMMOUCT4NCRONQF13ZB15SVLX4ZRUUNY0NTGZCF04SKYVS


In [78]:
pwd


'/resources'

In [79]:
ls

 Capstoneproject.ipynb
 Segmenting-and-Clustering-Neighborhoods-in-Toronto-Part-1-Copy1.ipynb
 Segmenting-and-Clustering-Neighborhoods-in-Toronto-Part-3.ipynb
 Untitled.ipynb
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 assesment1.ipynb
 data/
'final project'/
 labs/
 toronto.csv


In [45]:
# Read in the data 
import os 

df = pd.read_csv("/resources/final_project/Police_Department_Incidents_-_Previous_Year__2016_.csv",encoding='ISO-8859-1')
df.head(20)

#df=pd.read_csv("top50.csv",encoding='ISO-8859-1')
#pd.read_excel('Sheet1')

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000
5,160002869,ASSAULT,BATTERY,Friday,01/01/2016 12:00:00 AM,21:35,NORTHERN,NONE,1700 Block of BUSH ST,-122.426077,37.788019,"(37.788018555829, -122.426077177375)",16000286904134
6,160003130,OTHER OFFENSES,PAROLE VIOLATION,Saturday,01/02/2016 12:00:00 AM,00:04,SOUTHERN,"ARREST, BOOKED",MARY ST / HOWARD ST,-122.405721,37.780879,"(37.7808789360214, -122.405721454567)",16000313026150
7,160003259,NON-CRIMINAL,FIRE REPORT,Saturday,01/02/2016 12:00:00 AM,01:02,TENDERLOIN,NONE,200 Block of EDDY ST,-122.411778,37.783981,"(37.7839805592634, -122.411778295992)",16000325968000
8,160003970,WARRANTS,WARRANT ARREST,Saturday,01/02/2016 12:00:00 AM,12:21,SOUTHERN,"ARREST, BOOKED",4TH ST / BERRY ST,-122.393357,37.775788,"(37.7757876218293, -122.393357241451)",16000397063010
9,160003641,MISSING PERSON,FOUND PERSON,Friday,01/01/2016 12:00:00 AM,10:06,BAYVIEW,NONE,100 Block of CAMERON WY,-122.387182,37.720967,"(37.7209669615499, -122.387181635995)",16000364175000


In [46]:
# Reset the index and dropping the previous index
df = df.reset_index(drop=True)


In [47]:
# Shape of the data frame
df.shape

(150500, 13)

In [48]:
# View the top of the dataset 
df.head()

,IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


#### Change the column names

In [52]:
df.columns = ['Incidnt_Num', 'Category','Descript','Day_Of_Week','Date','Time','Pd_District','Resolution','Address','X','Y','Location','PDLD']
df.head()

,Incidnt_Num,Category,Descript,Day_Of_Week,Date,Time,Pd_District,Resolution,Address,X,Y,Location,PDLD
0,120058272,WEAPON LAWS,POSS OF PROHIBITED WEAPON,Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212120
1,120058272,WEAPON LAWS,"FIREARM, LOADED, IN VEHICLE, POSSESSION OR USE",Friday,01/29/2016 12:00:00 AM,11:00,SOUTHERN,"ARREST, BOOKED",800 Block of BRYANT ST,-122.403405,37.775421,"(37.775420706711, -122.403404791479)",12005827212168
2,141059263,WARRANTS,WARRANT ARREST,Monday,04/25/2016 12:00:00 AM,14:59,BAYVIEW,"ARREST, BOOKED",KEITH ST / SHAFTER AV,-122.388856,37.729981,"(37.7299809672996, -122.388856204292)",14105926363010
3,160013662,NON-CRIMINAL,LOST PROPERTY,Tuesday,01/05/2016 12:00:00 AM,23:50,TENDERLOIN,NONE,JONES ST / OFARRELL ST,-122.412971,37.785788,"(37.7857883766888, -122.412970537591)",16001366271000
4,160002740,NON-CRIMINAL,LOST PROPERTY,Friday,01/01/2016 12:00:00 AM,00:30,MISSION,NONE,16TH ST / MISSION ST,-122.419672,37.765050,"(37.7650501214668, -122.419671780296)",16000274071000


In [53]:
# View the information of the dataset 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150500 entries, 0 to 150499
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Incidnt_Num  150500 non-null  int64  
 1   Category     150500 non-null  object 
 2   Descript     150500 non-null  object 
 3   Day_Of_Week  150500 non-null  object 
 4   Date         150500 non-null  object 
 5   Time         150500 non-null  object 
 6   Pd_District  150499 non-null  object 
 7   Resolution   150500 non-null  object 
 8   Address      150500 non-null  object 
 9   X            150500 non-null  float64
 10  Y            150500 non-null  float64
 11  Location     150500 non-null  object 
 12  PDLD         150500 non-null  int64  
dtypes: float64(2), int64(2), object(9)
memory usage: 14.9+ MB


#### Total number of crimes in each District

In [55]:
df['Pd_District'].value_counts()

SOUTHERN      28445
NORTHERN      20100
MISSION       19503
CENTRAL       17666
BAYVIEW       14303
INGLESIDE     11594
TARAVAL       11325
TENDERLOIN     9942
RICHMOND       8922
PARK           8699
Name: Pd_District, dtype: int64

#### The total crimes per category


In [56]:
df['Descript'].value_counts()

GRAND THEFT FROM LOCKED AUTO             17741
LOST PROPERTY                             4596
AIDED CASE, MENTAL DISTURBED              4566
PETTY THEFT OF PROPERTY                   4416
MALICIOUS MISCHIEF, VANDALISM             4262
                                         ...  
IMPERSONATING FIRE DEPARTMENT MEMBER         1
PERMIT VIOLATION, SECOND-HAND DEALER         1
VIN SWITCH                                   1
GRAND THEFT BY PROSTITUTE                    1
DEATH, NON-MANSLAUGHTER AUTO ACCIDENT        1
Name: Descript, Length: 726, dtype: int64

### Pivoting the table to view the incident num. of each category and District

In [67]:
San_francisco = pd.pivot_table(df,values=['Incidnt_Num'],
                               index=['Pd_District'],
                               columns=['Category'],
                               aggfunc=np.sum,fill_value=0)
San_francisco.head()   

Incidnt_Num                                                     \
Category           ARSON       ASSAULT BAD CHECKS     BRIBERY      BURGLARY   
Pd_District                                                                   
BAYVIEW      11392647714  284979264583  652389707  3210396255   83685734659   
CENTRAL       4655785651  190699712987  481357567   482387767  103667946182   
INGLESIDE     3530917156  241846983101  321402745  1282909749   85708892344   
MISSION       7385678419  338812680662  321050661  1604927732  127449033579   
NORTHERN      4334203837  246669136734  641582641   641318772  129083605436   

                                                                          \
Category    DISORDERLY CONDUCT DRIVING UNDER THE INFLUENCE DRUG/NARCOTIC   
Pd_District                                                                
BAYVIEW             7874944446                  4336015810   52476255282   
CENTRAL             5138079312                  4987625894   33242599963   
INGLESIDE           2256333579                  5145326749   30649549861   
MISSION            27484461810                  9806798288  102563913353   
NORTHERN           19917413439                  6589419982   84608944978   

                                       ...                             \
Category     DRUNKENNESS EMBEZZLEMENT  ... SEX OFFENSES, NON FORCIBLE   
Pd_District                            ...                              
BAYVIEW       4336245669   3720517876  ...                  321538972   
CENTRAL       8316882542   3384065641  ...                          0   
INGLESIDE     2887179884   1133708299  ...                  472146905   
MISSION      16212644780   3892784868  ...                  482247221   
NORTHERN      9456570269   1455559003  ...                  481546645   

                                                                   \
Category    STOLEN PROPERTY     SUICIDE SUSPICIOUS OCC       TREA   
Pd_District                                                         
BAYVIEW          9472480227   642601559    98013972271          0   
CENTRAL         25074247628  1605769388    93255078739          0   
INGLESIDE        8982508359  2569678168    84742373819  160783865   
MISSION         16690895694   482233839   151885455500          0   
NORTHERN        19721723909  2249866197    96410347529          0   

                                                                    \
Category        TRESPASS     VANDALISM VEHICLE THEFT      WARRANTS   
Pd_District                                                          
BAYVIEW      20070513458  172203715716  173715631738   87934351241   
CENTRAL      27770039499  186849287837   77266541658   78478743594   
INGLESIDE    11885088084  123319296004  146958376115   49235563833   
MISSION      66187593645  176290617320  149764342353  172203673472   
NORTHERN     43041578904  195211100882  118705707615  100018857661   

                          
Category     WEAPON LAWS  
Pd_District               
BAYVIEW      49133371614  
CENTRAL      19606228082  
INGLESIDE    25182641774  
MISSION      44630942090  
NORTHERN     21030882508  

[5 rows x 39 columns]

In [68]:
# Reset the index
San_francisco.reset_index(inplace = True)

In [70]:
# Shape of the data set 
San_francisco.shape

(10, 41)